<a id='top'></a>

# CSCI 3202, Spring 2020
# Assignment 3
# Due:  Wednesday 4 March 2020 by 11:59 PM

<br>

### Your name: Angel Santana

<br>




**Some things that might be useful**


In [110]:
import pandas as pd
import numpy as np
import copy as cp
from scipy import stats
from math import floor
import matplotlib.pyplot as plt
import csv
from time import time

<br>

---

## Problem 1 (35 points):  Playing "intelligent" Tic-Tac-Toe

<img src="https://www.cookieshq.co.uk/images/2016/06/01/tic-tac-toe.png" width="150"/>

<a id='p2a'></a>

### (1a)   Defining the Tic-Tac-Toe class structure

Fill in this class structure for Tic-Tac-Toe using what we did during class as a guide.
* `moves` is a list of tuples to represent which moves are available. Recall that we are using matrix notation for this, where the upper-left corner of the board, for example, is represented at (1,1).
* `result(self, move, state)` returns a ***hypothetical*** resulting `State` object if `move` is made when the game is in the current `state`
* `compute_utility(self, move, state)` calculates the utility of `state` that would result if `move` is made when the game is in the current `state`. This is where you want to check to see if anyone has gotten `nwin` in a row
* `game_over(self, state)` - this wasn't a method, but it should be - it's a piece of code we need to execute repeatedly and giving it a name makes clear what task the piece of code performs. Returns `True` if the game in the given `state` has reached a terminal state, and `False` otherwise.
* `utility(self, state, player)` also wasn't a method earlier, but also should be.  Returns the utility of the current state if the player is X and $-1 \cdot$ utility if the player is O.
* `display(self)` is a method to display the current game `state`, You get it for free! because this would be super frustrating without it.
* `play_game(self, player1, player2)` returns an integer that is the utility of the outcome of the game (+1 if X wins, 0 if draw, -1 if O wins). `player1` and `player2` are functional arguments that we will deal with in parts **2b** and **2d**.

Some notes:
* Assume X always goes first.
* Do **not** hard-code for 3x3 boards.
* You may add attributes and methods to these classes as needed for this problem.

In [158]:
# Fill in as needed.
#used the code from the Notebook provided by Rachel
class State:
    def __init__(self, moves):
        self.toMove = 'X'
        self.utility = 0
        self.board = {}
        self.moves = moves
    
class TicTacToe:
        
    def __init__(self, nrow, ncol, nwin, nexp=0):
        self.nrow = nrow
        self.ncol = ncol
        self.nwin = nwin
#         self.nexp = nexp
        moves = [(row, col) for row in range(1, nrow+1) for col in range(1, ncol + 1)]
        self.state = State(moves)
        self.nexp = 0
    
    def result(self, move, state):
        '''
        What is the hypothetical result of move `move` in state `state` ?
        move  = (row, col) tuple where player will put their mark (X or O)
        state = a `State` object, to represent whose turn it is and form
                the basis for generating a **hypothetical** updated state
                that will result from making the given `move`
        '''
        #make sure move is not illegal
        if move not in state.moves:
            return state
        
        
        ns = cp.deepcopy(state) # create copy of the given state
        ns.utility = self.compute_utility(move, state)
        ns.board[move] = state.toMove
        ns.moves.remove(move)
        ns.toMove = ('O' if state.toMove == 'X' else 'X')
        return ns
    
    
    
    def compute_utility(self, move, state):
        '''
        What is the utility of making move `move` in state `state`?
        If 'X' wins with this move, return 1;
        if 'O' wins return -1;
        else return 0.
        '''
        row, col = move
        player = state.toMove
        
        #create a board with move executed
        board = cp.deepcopy(state.board)
        board[move] = player
        
        #get all the possible cases that a player can move 
        
        #first check for the row type win
        threeRow = 0
        for c in range(1, self.ncol+1):
            threeRow += board.get((row, c)) == player
        
        #check for three in a column 
        threeCol = 0
        for r in range(1, self.nrow + 1):
            threeCol += board.get((r, col)) == player
        
        #check for diagnals (first from top left to bottom right)
        diagnal = 0
        for r in range(row, 0, -1):
            diagnal += board.get((r, col - (row-r))) == player
        for r in range(row + 1, self.nrow+1):
            diagnal += board.get((r, col - (row-r))) == player
        
        #check for the other diagnol from the bottom left to top right
        other_diagnal = 0
        for r in range(row, 0, -1):
            other_diagnal += board.get((r, col + (row-r))) == player
        for r in range(row + 1, self.nrow+1):
            other_diagnal += board.get((r, col + (row-r))) == player
        
        
        #once we check if there is a win
        # we want to return the win
        if self.nwin in [threeRow, threeCol, diagnal, other_diagnal]:
            if player == 'X': return 1
            else: return -1
        else: return 0

        

    def game_over(self, state):
        '''game is over if someone has won (utility!=0) or there
        are no more moves left'''
        return state.utility != 0 or len(state.moves) == 0
        
           

    
    def utility(self, state, player):
        '''Return the value to player; 1 for win, -1 for loss, 0 otherwise.'''
        if player == 'X':
            return state.utility
        else: return -state.utility
        
                
        
        
    def display(self):
        for row in range(1, self.nrow+1):
            for col in range(1, self.ncol+1):
                print(self.state.board.get((row, col), '.'), end=' ')
            print()
        
    def play_game(self, player1, player2):
        '''Play a game of tic-tac-toe!'''
        # create a limit for turns just in case someone gets skipped by accident 
        limitTurn = self.nrow*self.ncol
        turn = 0
        while turn < limitTurn:
            for player in [player1, player2]:
                turn =  turn + 1
                move = player(self)
                self.state = self.result(move, self.state)
                if self.game_over(self.state):
                    self.display()
                    return self.state.utility
    '''Using this to find the percentage without the self.display'''
    def play_game2(self, player1, player2):
        '''Play a game of tic-tac-toe!'''
        # create a limit for turns just in case someone gets skipped by accident 
        limitTurn = self.nrow*self.ncol
        turn = 0
        while turn < limitTurn:
            for player in [player1, player2]:
                turn =  turn + 1
                move = player(self)
                self.state = self.result(move, self.state)
                if self.game_over(self.state):
                    return self.state.utility






<br>


### (1b) Define a random player

Define a function `random_player` that takes a single argument of the `TicTacToe` class and returns a random move out of the available legal moves in the `state` of the `TicTacToe` game.

In your code for the `play_game` method above, make sure that `random_player` could be a viable input for the `player1` and/or `player2` arguments.

In [159]:
#Used code from Notebook provided by Rachel
def random_player(game):
    '''A player that chooses a legal move at random out of all
    available legal moves in Tic-Tac-Toe state argument'''
    actions = game.state.moves
    return actions[np.random.randint(low=0, high=len(actions))]



<br>


### (1c) What about playing randomly on different-sized boards?

What does the long-term win percentage appear to be for the first player in a 4x4 Tic-Tac-Toe tournament, where 4 marks must be connected for a win?  Support your answer using a simulation and printed output, similar to **2b**.

**Also:** The win percentage should have changed substantially. Did the decrease in wins turn into more losses for the first player or more draws? Write a few sentences explaining the behavior you observed.  *Hint: think about how the size of the state space has changed.*

In [160]:
#Test Cases
iterations = 100
wins = 0
losses = 0
draws = 0

for i in range(iterations):
    t3 = TicTacToe(3, 3, 3)
    '''this play_game2 is an implementation without the self.display()'''
    show1 = t3.play_game2(random_player, random_player)
    '''This commented code shows the simulatons'''
#     show = t3.play_game2(random_player, random_player)
    if show1 == 1:
        wins += 1
    elif show1 == -1:
        losses += 1
    else: 
        draws += 1
        
print("Winning Percentage for 3x3:", wins/iterations*100, "%")

wins1 = 0
losses1 = 0
draws1 = 0
print("")
for i in range(iterations):
    t4 = TicTacToe(4, 4, 4)
    '''this play_game2 is an implementation without the self.display()'''
    show = t4.play_game2(random_player, random_player)
    '''This commented code shows the simulatons'''
#     show = t4.play_game(random_player, random_player)
    if show == 1:
        wins1 += 1
    elif show == -1:
        losses1 += 1
    else: 
        draws1 += 1
        
print("Winning Percentage for 4x4:", wins1/iterations * 100, "%")


Winning Percentage for 3x3: 59.0 %

Winning Percentage for 4x4: 28.999999999999996 %


After running running 100 simulations for the 4x4 **Tic-Tac-Toe**, the win percentage did change substantially. In a 3x3, player 2 has the oppurtunity to close out player 1, causing a loss or a draw with less moves, however, in a 4x4, it takes alot more moves, and it makes a little easier for player2 to do the same as in a 3x3 because there are more possible actions to not let player 1 win

<br>


### (1d) Define an alpha-beta player

Alright. Let's finally get serious about our Tic-Tac-Toe game.  No more fooling around!

Craft a function called `alphabeta_player` that takes a single argument of a `TicTacToe` class object and returns the minimax move in the `state` of the `TicTacToe` game. As the name implies, this player should be implementing alpha-beta pruning as described in the textbook and lecture.

Note that your alpha-beta search for the minimax move should include function definitions for `max_value` and `min_value` (see the aggressively realistic pseudocode from the lecture slides).

In your code for the `play_game` method above, make sure that `alphabeta_player` could be a viable input for the `player1` and/or `player2` arguments.

In [161]:
#Used implementation from Notebook provided by Rachel
def alphabeta_player(game):
    return alphabeta_search(game)

def alphabeta_search(game):
    #define alpha, beta
    alpha = float("-inf")
    beta = float("inf")
    
    #define what function to use depending on the move
    option = max_value if game.state.toMove == 'X' else min_value
    
    val, abMove = option(game, game.state, alpha, beta)
    return abMove
#Used slides from class
#and used the following site for help
#https://www.geeksforgeeks.org/minimax-algorithm-in-game-theory-set-4-alpha-beta-pruning/
def max_value(game, state, alpha, beta):
    #check to see if game is already over
    if(game.game_over(state)):
        return (state.utility, None)
    
    val = float("-inf") #the best case
    bestOption = None #check for the best option
    for mv in state.moves: #for each child node
        game.nexp += 1 #if any states expanded, keep track
        #choose the best value
        chooseBest = max(val, min_value(game, game.result(mv, state), alpha, beta)[0])
        #make the best child node the best option
        if val != chooseBest:
            val = chooseBest
            bestOption = mv
        #used for pruning 
        if beta <= val:
            bestOption = mv
            return(val, bestOption)
        alpha = max(val, alpha)
    #returns the best option
    return(alpha, bestOption)
        
def min_value(game, state, alpha, beta):
    #check to see if game is already over
    if(game.game_over(state)):
        return (state.utility, None)
    
    val = float("inf")#initialize best case
    bestOption = None#intialize best option
    for mv in state.moves:#for each child in 
        game.nexp += 1 #if any states are expanded
        #choose the best option from any min value
        chooseBest = min(val, max_value(game, game.result(mv, state), alpha, beta)[0])
        #make best child node the best option
        if val != chooseBest:
            val = chooseBest
            bestOption = mv
        #used for pruning 
        if alpha >= val:
            bestOption = mv
            return(val, bestOption)
        beta = min(val, beta)
    #return the best option
    return(beta, bestOption)
    
 

Verify that your alpha-beta player code is working appropriately through the following tests, using a standard 3x3 Tic-Tac-Toe board. Run **only 10 games for each test**, and track the number of wins, draws and losses.

1. An alpha-beta player who plays first should never lose to a random player who plays second.
2. A random player who plays first should never win to an alpha-beta player who plays second.
3. Two alpha-beta players should always draw.

**Nota bene:** Test your code with fewer games between the players to start, because the alpha-beta player will require substantially more compute time than the random player.  This is why I only ask for 10 games, which still might take a minute or two. 

In [163]:
#test 1 for alphabeta player and random player
abp_won = 0
abp_draw = 0
abp_loss = 0
iterations = 10
for it in range(iterations):
    ttt = TicTacToe(3, 3, 3)
    score = ttt.play_game2(alphabeta_player, random_player)
    if score == 1: #for a win
        abp_won += 1
    elif score == -1: #for a loss
        abp_loss += 1
    else:#for a draw
        abp_draw += 1
print("Alphabeta Player as player 1 won: ", abp_won/iterations * 100, "%")
print("Alphabeta Player as player 1 drew: ", abp_draw/iterations * 100, "%")
print("Alphabeta Player as player 1 lost: ", abp_loss/iterations * 100, "%")

print("---------------------------------------------------")

#test 2 for alphabeta player and random player but alphabeta player as p2
abp2_won = 0
abp2_draw = 0
abp2_loss = 0
iterations2 = 10
for it in range(iterations2):
    ttt = TicTacToe(3, 3, 3)
    score = ttt.play_game2(random_player, alphabeta_player)
    if score == -1: #for a win
        abp2_won += 1
    elif score == 1: #for a loss
        abp2_loss += 1
    else:#for a draw
        abp2_draw += 1
print("Alphabeta Player as player 2 won: ", abp2_won/iterations2 * 100, "%")
print("Alphabeta Player as player 2 drew: ", abp2_draw/iterations2 * 100, "%")
print("Alphabeta Player as player 2 lost: ", abp2_loss/iterations2 * 100, "%")
        

print("---------------------------------------------------")

#test 3 for alphabeta player and alphabeta player
abp3_won = 0
abp3_draw = 0
abp3_loss = 0
iterations3 = 10
for it in range(iterations):
    ttt = TicTacToe(3, 3, 3)
    score = ttt.play_game2(alphabeta_player, alphabeta_player)
    if score == 1: #for a win
        abp3_won += 1
    elif score == -1: #for a loss
        abp3_loss += 1
    else:#for a draw
        abp3_draw += 1
print("Alphabeta Player as both players won: ", abp3_won/iterations3 * 100, "%")
print("Alphabeta Player as both players drew: ", abp3_draw/iterations3 * 100, "%")
print("Alphabeta Player as both players lost: ", abp3_loss/iterations3 * 100, "%")
        




Alphabeta Player as player 1 won:  100.0 %
Alphabeta Player as player 1 drew:  0.0 %
Alphabeta Player as player 1 lost:  0.0 %
---------------------------------------------------
Alphabeta Player as player 2 won:  70.0 %
Alphabeta Player as player 2 drew:  30.0 %
Alphabeta Player as player 2 lost:  0.0 %
---------------------------------------------------
Alphabeta Player as both players won:  0.0 %
Alphabeta Player as both players drew:  100.0 %
Alphabeta Player as both players lost:  0.0 %


<br>


### (1e) What has pruning ever done for us?

Calculate the number of number of states expanded by the minimax algorithm, with and without pruning, to determine the minimax decision from the initial 3x3 Tic-Tac-Toe board state.  This can be done in many ways, but writing out all the states by hand is **not** one of them (as you will find out!).

Write a sentence or two, commenting on the difference in number of nodes expanded by each search.

In [116]:
ttt = TicTacToe(3, 3, 3)
alphabeta_player(ttt)
print("Number of states expanded: ", ttt.nexp)


Number of states expanded:  18296


In [117]:
'''For these functions we take out the parts where pruning happens'''
def alphabeta_player2(game):
    return alphabeta_search2(game)

def alphabeta_search2(game):
    #define alpha, beta
    alpha = float("-inf")
    beta = float("inf")
    
    #define what function to use depending on the move
    option = max_value2 if game.state.toMove == 'X' else min_value2
    
    val, abMove = option(game, game.state, alpha, beta)
    return abMove

def max_value2(game, state, alpha, beta):
    #check to see if game is already over
    if(game.game_over(state)):
        return (state.utility, None)
    
    val = float("-inf") #the best case
    bestOption = None #check for the best option
    for mv in state.moves: #for each child node
        game.nexp += 1 #if any states expanded, keep track
        #choose the best value
        chooseBest = max(val, min_value2(game, game.result(mv, state), alpha, beta)[0])
        #make the best child node the best option
        if val != chooseBest:
            val = chooseBest
            bestOption = mv
#         #used for pruning 
#         if beta <= val:
#             bestOption = mv
#             return(val, bestOption)
        alpha = max(val, alpha)
    #returns the best option
    return(alpha, bestOption)
        
def min_value2(game, state, alpha, beta):
    #check to see if game is already over
    if(game.game_over(state)):
        return (state.utility, None)
    
    val = float("inf")#initialize best case
    bestOption = None#intialize best option
    for mv in state.moves:#for each child in 
        game.nexp += 1 #if any states are expanded
        #choose the best option from any min value
        chooseBest = min(val, max_value2(game, game.result(mv, state), alpha, beta)[0])
        #make best child node the best option
        if val != chooseBest:
            val = chooseBest
            bestOption = mv
#         #used for pruning 
#         if alpha >= val:
#             bestOption = mv
#             return(val, bestOption)
        beta = min(val, beta)
    #return the best option
    return(beta, bestOption)
    
 

In [118]:
tt2 = TicTacToe(3, 3, 3)
alphabeta_player2(tt2)
print("Number of States without pruning: ", tt2.nexp)

Number of States without pruning:  549945


We can see that the differene of expanding nodes is significant. What pruning helps with is that it eliminates any chance of looking at any states that have no good option down that path, as we saw in the exam, if it isn't viable, it won't help in getting the best move to be able to win. 

<br>

---

## Problem 2 (30 points):  Uncertainty

### (2a) 

Suppose you are deciding when to arrive at a party. There is some optimal time to arrive when the loss you feel, as measured by _awkwardness_, is minimized at 0. That is, at some particular time, it is not awkward at all to show up to the party. The awkwardness (loss) increases as you arrive too early or too late relative to this optimal time. What is a suitable loss function, $L(d, x)$, to model this situation? Include definitions for $d$ and $x$, consistent with the examples from this class. Use this loss function this weekend when you go out.

- What we can do is set up a linear equation.
    - We will let $x$ = state variable (the time you left)
    - We will let $d$ = decision to make (time it takes to arrive to the part)
- We know that the loss function is to be able to minimize, and be able to arrive at an optimal time, not to early or late. 
- We cant set up our function as follows: where $L(d, x)$ = $(d-x)^2$
- If time doesn't get close to being optimal and heads in the direction of either arriving early or arriving late, then with the loss function increasing in a linear matter, this will be equal to being that awkward person of arriving early or late. 

### (2b)

Suppose we have a situation where loss is given by the function $L(d, x) = 2(d-x)^2$. Set up, simplify, and evaluate integral(s) for the expected loss, $E_x[L(d, x)]$, where your prior beliefs regarding $x$ follow the distribution $f(x)$ given below. You may assume $f(x)=0$ for values of $x$ outside of the interval $[0, 3]$.

f(x) =  \begin{cases} 
      1/2 & 0 \leq x <1 \\
      3/8 & 1 \leq x <2 \\
      1/8 & 2 \leq x \leq 3 
   \end{cases}


- We set up our function: 
    - $\int_{x}^{} 2(d-x)^2 dx$
- Now, we will evaluate from 0 - 1, then 1 - 2, then lastly 2 - 3
    - $\int_{0}^{1} 2(d-x)^2 \cdot \frac{1}{2}dx + \int_{1}^{2} 2(d-x)^2 \cdot \frac{3}{8}dx + \int_{2}^{3} 2(d-x)^2 \cdot \frac{1}{8}dx$
- Which will evaluate to:
    - $2d^2 - \frac{9d}{2} + \frac{11}{3}$

### (2c) 

Suppose our expected loss is represented by the function $E_x[(L(d,x)]=(2-d)^2+2$, and our prior beliefs regarding $x$ are given by the distribution $f(x)$ from part b.

- Calculate Bayes' Decision, $d_{Bayes}$.

- Calculate the Expected Value of Including Uncertainty, EVIU. Suppose that if we ignore uncertainty, our best guess for what decision to make is the median of $x$ (under our prior $f(x)$).

--------------
- Calculating Bayes' Decision:
    - This value can be found when the function is minimized:
        - so we want to get a value for $d$ to make the function equal to 0
        - we get the following:
            - $= (2-2)^2+2$
            - $= 0 + 2$
            - $\approx{0}$
    - Hence we get $d_{Bayes} = 2$
---
- Calculating EVIU
    - We set up out function to be:
        - $ \int_{x}{} [L(d_{iu},x) - L(d_{Bayes},x)]f(x) dx$
        - Which is:
            - $=(2-d_{iu})^2+2 - (2-d_{Bayes})^2-2$
    - We know that $d_{iu}$ is where mass is half of the probability, so looking back at our $f(x)$ function, we see that between 0 and 1 it is $\frac{1}{2}$, and between 1 and 3 it is $\frac{4}{8}$, thus we can say that $d_{iu}$ = 1
    - Given the information, we can plug in the values from $d_{Bayes}$ and $d_{iu}$
        - We get:
            - $=(2-1)^2+2 - (2-2)^2-2$
            - which will result in a value of $\textbf{1}$

<br>

## Problem 3 (35 points): Maximizing some objective function with a Genetic Algorithm

Suppose we are trying to figure out a cookie recipe, but can't quite remember how much of each ingredient we need.

So we want to maximize the following objective function corresponding to how close we are to this recipe:

* 3/4 cup granulated sugar (36 tsp)
* 3/4 cup packed brown sugar  (36 tsp)
* 1 cup butter (48 tsp)
* 1 teaspoon vanilla (1 tsp)
* 1 egg
* 2 1/4 cups flour (108 tsp)
* 1 teaspoon baking soda (1 tsp)
* 1/2 teaspoon salt (0.5 tsp)
* 1 package (12 ounces) chocolate chips (2 cups) (96 tsp)

In [120]:
target = [36, 36, 48, 1, 1, 108, 1, 0.5, 96]

An example starting state for a member of our population might look like: $state = [30, 30, 40, 1, 0, 100, 0.5, 0.5, 100]$

### (3a) 

Write an objective function `def recipe_success(state)` that takes a single argument state, and returns the objective function value (fitness) of the state. The objective function should be maximized when a state reaches the target. You could for example define the fitness score of a particular state based on how far away each entry is from the target recipe.

In [154]:
#using Notebook provided by Rachel "genetic_algorithm_niceexample"
def recipe_success(state):
    # Your code here.
    global target
    fitness = 10
    for s, t in zip(state, target):
        if s != t:
            fitness -=1
    return fitness
    
    

### (3b) 

Using our in class notebook "CSCI3202_GeneticAlgorithm.ipynb" as your guide, write a genetic algorithm that starts with a population of 10 randomly generated "recipes/states/members" and uses the objective function you wrote in **(3a)** to hopefully hit the target after a certain number of generations. 

Key components of your code:
- Generate the initial population randomly from numbers between 0 and 108 (half step intervals might be helpful since the recipe requires 1/2 tsp salt)
- Allow for mutations in your population with an overall probability of mutation set to p = 0.1
- Choose 2 "parents" in the generation of each "child"
- Choose a random split point at which to combine the two "parents"
- Run the algorithm for 200 iterations ("generations"). Do you hit your target?

In [155]:
#using the NoteBook geneticalgorithm that was provided by Rachel
class problem:
    def __init__(self, initial_population, objective_function, mutation_probability, fitness_goal):
        '''
        initial_population = list of lists; each sub-list is a dna string for a population member
        objective_function = objective function to maximize
        mutation_probability = probability that any given child has a mutation
        fitness_goal = fitness goal to achieve (stopping criterion, once member reaches this)
        '''
        self.population = initial_population
        self.initial_population = initial_population
        self.objective_function = objective_function
        self.p_mutate = mutation_probability
        self.n_pop = len(initial_population)
        self.n_dna = len(initial_population[0])
        self.fitness_goal = fitness_goal

    def fitness(self):
        '''
        calculate each population member's probability of being selected for
        reproduction based on performance on objective function
        '''
        performance = []
        for k in range(self.n_pop):
            performance.append(self.objective_function(self.population[k]))
        total = sum(performance)
        p_reproduce = [perf/sum(performance) for perf in performance]
        return p_reproduce
        
    def reproduce(self, parent1, parent2):
        # last DNA snippet from parent1:
        split = np.random.randint(low=1, high=self.n_dna)
        child = parent1[:split] + parent2[split:]
        return child

    def mutate(self, child): #changed from original notebook to be able to satisfy our goal recipe
        # which gene to mutate?
        gene = np.random.randint(low=0, high=self.n_dna)
        change = np.random.choice(np.arange(0, 109.5, .5))
        child[gene] = change
        return child
    
   
   

In [156]:
def genetic_algorithm(problem, n_iter):
    for t in range(n_iter):
        
        new_generation = []
        
        for k in range(problem.n_pop):
            
            # select for reproduction
            p_reproduce = problem.fitness()
            ind_parents = np.random.choice(range(0,problem.n_pop), size=2, p=p_reproduce, replace=False)
            parent1, parent2 = problem.population[ind_parents[0]], problem.population[ind_parents[1]]
            
            # reproduce
            child = problem.reproduce(parent1, parent2)
            
            # mutate
            l_mutate = np.random.choice([True, False], p=[problem.p_mutate, 1-problem.p_mutate])
            if l_mutate:
                child = problem.mutate(child)
            
            # add to new generation
            new_generation.append(child)
        
        # set problem.population = new generation
        problem.population = new_generation
        
        # exit criterion check
        performance = [problem.objective_function(member) for member in problem.population]
        
        best_member = max(zip(performance, problem.population))
        
        if best_member[0] >= problem.fitness_goal:
            return best_member

    print('reached n_iter')

    return False

### (3c)

Report the following:
- How many generations did it take to hit the goal?
- If you change the initial population size to 100, does that change the number of generations it takes to achieve the goal recipe?
- If you change the probability of mutation to 0.2, does that affect the number of generations it takes to achieve the goal recipe?

In [157]:
# Your code here.
initial_population = []
for i in range(100):
    initial_population.append(list(np.random.choice(np.arange(0, 108.5, .5), size = 9)))
# print(initial_population)
genetic_problem = problem(initial_population=initial_population, 
                          fitness_goal=10, 
                          mutation_probability=0.2, 
                          objective_function=recipe_success)
out = genetic_algorithm(genetic_problem, 200)
print(out)

reached n_iter
False


Because we are randomizing our choice in the list, we see that it has reached n_iter, showing that it didn't achieve the goal recipe.